In [144]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed,MaxPooling1D, Flatten, Conv1D,Conv2D,Dropout, MaxPooling2D, GRU # for creating layers inside the Neural Network
from keras.optimizers import Adam , SGD
from keras import metrics
from keras.utils import plot_model
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import numpy.ma as ma
# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling
from sklearn.model_selection import train_test_split
# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version
from matplotlib import pyplot as plt

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar


Tensorflow/Keras: 2.9.0
pandas: 1.4.2
numpy: 1.22.4
sklearn: 1.1.1
plotly: 5.9.0


##### Read from file and establish dataframe and future functions

In [145]:
df=pd.read_csv('InSAR_data_south/displacement/export_dataframe1.csv')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y%m%d')
df=df.dropna(axis=0, how='all')#drop full nan rows

##### Remove next line to include all data

In [146]:
df=df.iloc[200880:200890] #cuts data to long 120-119 approx
#df=df.iloc[200000:200050]

df

2014-11-08  2014-12-02  2014-12-26  2015-02-12  \
Longitude Latitude                                                   
-120.4087 36.9214          0.0     -7.5666     -1.1857      1.2461   
          36.9234          0.0     -7.5248     -1.1719      1.3339   
          36.9254          0.0     -7.5343     -1.3051      1.1993   
          36.9274          0.0     -7.4905     -1.2349      1.2690   
          36.9294          0.0     -7.4614     -1.2258      1.2765   
          36.9314          0.0     -7.4634     -1.1660      1.3789   
          36.9334          0.0     -7.4255     -1.1440      1.4084   
          36.9354          0.0     -7.4360     -1.1476      1.4514   
          36.9374          0.0     -7.4381     -1.1158      1.5108   
          36.9394          0.0     -7.4429     -1.0727      1.6068   

                    2015-03-08  2015-04-01  2015-04-25  2015-05-19  \
Longitude Latitude                                                   
-120.4087 36.9214      -2.0524      2.4467     -2.9248   -0.064559   
          36.9234      -1.9606      2.5024     -2.8837   -0.134060   
          36.9254      -2.0983      2.3195     -3.1164   -0.511280   
          36.9274      -2.0480      2.3709     -3.0573   -0.513620   
          36.9294      -2.0475      2.3984     -2.9988   -0.444090   
          36.9314      -1.9943      2.4414     -2.8526   -0.350980   
          36.9334      -1.9439      2.5082     -2.7706   -0.238070   
          36.9354      -1.9075      2.5591     -2.7901   -0.236760   
          36.9374      -1.8549      2.5910     -2.7065   -0.232510   
          36.9394      -1.7088      2.6848     -2.6101   -0.073500   

                    2015-06-12  2015-07-06  ...  2018-10-18  2018-10-30  \
Longitude Latitude                          ...                           
-120.4087 36.9214      0.90346     -2.5683  ...     -27.043     -24.400   
          36.9234      0.83566     -2.6301  ...     -28.755     -26.130   
          36.9254      0.44051     -3.0954  ...     -31.703     -29.067   
          36.9274      0.40750     -3.1829  ...     -34.750     -32.103   
          36.9294      0.47399     -3.0762  ...     -35.470     -32.829   
          36.9314      0.57990     -2.9256  ...     -34.738     -32.112   
          36.9334      0.67341     -2.7841  ...     -33.479     -30.837   
          36.9354      0.66471     -2.7504  ...     -33.379     -30.735   
          36.9374      0.65473     -2.7714  ...     -33.089     -30.418   
          36.9394      0.67181     -2.7176  ...     -33.465     -30.738   

                    2018-11-11  2018-11-17  2018-11-23  2018-12-05  \
Longitude Latitude                                                   
-120.4087 36.9214      -25.155     -25.844     -29.027     -30.252   
          36.9234      -26.901     -27.640     -30.745     -31.948   
          36.9254      -29.805     -30.507     -33.684     -34.911   
          36.9274      -32.770     -33.508     -36.727     -37.891   
          36.9294      -33.479     -34.205     -37.386     -38.456   
          36.9314      -32.788     -33.488     -36.724     -37.750   
          36.9334      -31.484     -32.195     -35.461     -36.450   
          36.9354      -31.377     -32.099     -35.311     -36.275   
          36.9374      -31.089     -31.807     -34.990     -35.891   
          36.9394      -31.458     -32.147     -35.291     -36.132   

                    2018-12-17  2018-12-29  2019-01-10  2019-01-22  
Longitude Latitude                                                  
-120.4087 36.9214      -27.912     -26.146     -26.877     -24.533  
          36.9234      -29.557     -27.866     -28.581     -26.237  
          36.9254      -32.592     -30.910     -31.643     -29.279  
          36.9274      -35.689     -34.034     -34.815     -32.406  
          36.9294      -36.252     -34.634     -35.449     -32.931  
          36.9314      -35.539     -33.964     -34.795     -32.236  
          36.9334      -34.290     -32.721  

In [147]:
df2=pd.read_csv('InSAR_data_south/displacement/groundwater.csv')
df2=df2.set_index([df2.columns[0],df2.columns[1]])
df2.columns=pd.to_datetime(df2.columns, format='%Y-%m-%d')
df2=df2.dropna(axis=0, how='all')#drop full nan rows
for y in range(df.columns.size):# includes all dates and adds nans
    if df.columns[y] != df2.columns[y]: #if column is misssing insert it
        df2.insert(loc=y,column=df.columns[y],value=np.nan*11286,allow_duplicates=False)
df2.index[0]
type(df2.index[0])


tuple

In [148]:
for x in range(df2.index.size): #for each row
    curr=np.nan
    for y in range(df2.columns.size): #for each value in the row
        n=0
        while np.isnan(curr): # if saved value is empty
            curr=df2.iloc[x].values[n] #look for next actual value
            n+=1
        if np.isnan(df2.iloc[x].values[y]):#if data is nan then replace with previous curr value
            df2.iat[x,y]=curr
            #df2.iloc[x].values[y]=curr
        else:
            curr= df2.iloc[x].values[y]
                

df2

,,2014-11-08,2014-12-02,2014-12-26,2015-02-12,2015-03-08,2015-04-01,2015-04-25,2015-05-19,2015-06-12,2015-07-06,...,2018-10-18,2018-10-30,2018-11-11,2018-11-17,2018-11-23,2018-12-05,2018-12-17,2018-12-29,2019-01-10,2019-01-22
Longitude,Latitude,,,,,,,,,,,,,,,,,,,,,
-120.9486,37.6316,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,...,75.653992,75.813688,75.973384,76.0,76.0,76.0,76.0,76.0,76.0,76.0
-120.8926,36.7114,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,...,75.200000,75.200000,75.200000,75.2,75.2,75.2,75.2,75.2,75.2,75.2
-120.8846,36.6994,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,...,75.200000,75.200000,75.200000,75.2,75.2,75.2,75.2,75.2,75.2,75.2
-120.8746,36.6874,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,72.248964,...,75.200000,75.200000,75.200000,75.2,75.2,75.2,75.2,75.2,75.2,75.2
-120.1368,35.6991,141.123387,141.123387,141.123387,141.123387,141.123387,141.123387,141.123387,141.123387,141.123387,141.123387,...,150.898190,151.000000,151.000000,151.0,151.0,151.0,151.0,151.0,151.0,151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-119.0351,36.3933,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,...,16.978182,17.000000,17.000000,17.0,17.0,17.0,17.0,17.0,17.0,17.0
-119.0271,36.3793,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,...,16.978182,17.000000,17.000000,17.0,17.0,17.0,17.0,17.0,17.0,17.0
-119.0571,36.4493,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,17.409091,...,16.978182,17.000000,17.000000,17.0,17.0,17.0,17.0,17.0,17.0,17.0


# Timestep

In [149]:
timestep=10
# 1 2 3 4 5 6 7 ...
# w/ timestep of 2
# [[1,2][2,3][3,4]

# Shaping function (remove start and step for first for loop to include all data)

In [150]:

def shaping2(datain,datain2, timestep):
    #print(arr)
    cnt=0
    for row in range(len(datain2.index)): #picks a row at every iteration, allows to reduction of input and inclusion of multiple time series, remove start and step to run on full dataset 
    # Convert input dataframe to array and flatten
        #datain.iloc[row].fillna(datain.iloc[row].mean).to_numpy().flatten()
        #if datain2.index[row] not in datain.index:
        if datain2.index[row] not in datain.index:
            #print("Not found", cnt)
            continue
        print(datain.loc[datain2.index[row]])
        print("found")
        arr=datain.loc[datain2.index[row]].to_numpy().flatten() # flatten row
        arr=np.where(np.isnan(arr), ma.array(arr, mask=np.isnan(arr)).mean(), arr) 
        arr2=datain2.iloc[row].to_numpy().flatten()
        arr3=np.concatenate((arr,arr2)).reshape(2,110)
        
        
        for mth in range(0, len(datain2.columns)-(2*timestep)+1): # Define range lenght of the dates - 2* amount of timesep?? +1
            cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
            X_start=mth # Start month for inputs of each sample
            X_end=mth+timestep # End month for inputs of each sample
            Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
            Y_end=mth+2*timestep # End month for targets of each sample.  
            
            # Assemble input and target arrays containing all samples
            if cnt==1:
                X_comb=arr3[:,X_start:X_end]
                Y_comb=arr3[0][Y_start:Y_end]
            else: 
                X_comb=np.append(X_comb, arr3[:,X_start:X_end])
                Y_comb=np.append(Y_comb, arr3[:1,Y_start:Y_end])
    
    # Reshape input and target arrays
    X_out=np.reshape(X_comb, (cnt, timestep, 2))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

In [151]:
# print(df2.index.values[0])
# #arr=df.loc[df.index[0]]
# if df2.index.values in df.index.values:
#     print(True)
train, test = train_test_split(df, test_size=0.2, random_state=64, shuffle=False)
train2, test2 = train_test_split(df2, test_size=0.2, random_state=64, shuffle=False)
X_train, Y_train=shaping2(datain=train,datain2=train2, timestep= timestep )


UnboundLocalError: local variable 'X_comb' referenced before assignment

##### Step 2 - Split and Shape data